In [1]:
import pandas as pd
import numpy as np


In [2]:
df1= pd.read_excel('Dataset.xlsx') 
df=df1.sample(frac = 0.50)

In [3]:
df.head()

,SL,Emotion,Text,Year,Racial Biases,Hatred Percentage,Depression Percentage
7894,7895,disgust,বিনাভোটে নির্বাচিত সরকারের মন্ত্রীর মুখে গনতন্...,2017,NaN,NaN,NaN
9038,9039,disgust,এ ধরণের নৃশংস হত্যাকান্ড মোটেও সমর্থনযোগ্য নয়।,2017,NaN,NaN,NaN
5575,5576,surprise,আবারও নিগারের সেঞ্চুরি; বাঘিনীদের টানা চতুর্থ জয়,2016,NaN,NaN,NaN
32804,32805,sad,"তার প্রধান কারণই হল, শিক্ষা এবং সংস্কৃতিতে সাধ...",2021,NaN,NaN,NaN
3019,3020,angry,এইসমস্ত নরাধম দাগী যুদ্ধাপরাধীদের দ্রুত ঝুলানো...,2016,NaN,NaN,NaN


In [4]:
df.tail()

,SL,Emotion,Text,Year,Racial Biases,Hatred Percentage,Depression Percentage
34614,34615,disgust,"তাদের কথা হলো, সরকার নিজেদের দেশের মাত্র একটি...",2021,NaN,NaN,NaN
22673,22674,sad,ঠাকুর ঘুমাই ছিলরে ভাই।,2019,NaN,NaN,NaN
17750,17751,surprise,কাকে কি বলবো.....তবে জনাব শাকিব কাজটা ঠিক করেন নি,2018,NaN,NaN,NaN
26119,26120,fear,"সরকারের গদি হারানোর খুব ভয় এ জন্য এই চালাকি, য...",2020,NaN,NaN,NaN
35946,35947,surprise,ভারতের সেনাপ্রধান পাঁচ দিনের সফরে ঢাকায়,2021,NaN,NaN,NaN


In [5]:
df["Emotion"].value_counts()

angry       3068
surprise    3010
sad         2998
disgust     2982
happy       2977
fear        2965
Name: Emotion, dtype: int64

In [6]:
class_names = ['disgust','angry','happy','surprise','fear','sad']

In [7]:
encoding = {
    'disgust': 0, 
    'angry' :1,
    'happy':2,
    'surprise':3,
    'fear':4,
    'sad':5
}

In [8]:
text = df['Text'].values.tolist()
emotion= df['Emotion'].tolist()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2, random_state=42)

In [10]:
df["Emotion"].value_counts()

angry       3068
surprise    3010
sad         2998
disgust     2982
happy       2977
fear        2965
Name: Emotion, dtype: int64

In [11]:
import ktrain
from ktrain import text
MODEL_NAME = 'sagorsarker/bangla-bert-base'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [12]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 16
	95percentile : 44
	99percentile : 67


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 45
	99percentile : 64


In [13]:
model = t.get_classifier()

In [14]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [15]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
2400/2400 [==============================] - 26804s 11s/step - loss: 1.5556 - accuracy: 0.3705 - val_loss: 1.2421 - val_accuracy: 0.5619
Epoch 2/5
2400/2400 [==============================] - 26855s 11s/step - loss: 0.9671 - accuracy: 0.6615 - val_loss: 0.9203 - val_accuracy: 0.6814
Epoch 3/5
2400/2400 [==============================] - 26535s 11s/step - loss: 0.5726 - accuracy: 0.8097 - val_loss: 0.7728 - val_accuracy: 0.7594
Epoch 4/5
2400/2400 [==============================] - 26993s 11s/step - loss: 0.2805 - accuracy: 0.9085 - val_loss: 0.7464 - val_accuracy: 0.7917
Epoch 5/5
2400/2400 [==============================] - 26778s 11s/step - loss: 0.1350 - accuracy: 0.9534 - val_loss: 0.8114 - val_accuracy: 0.7969


In [16]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

     disgust       0.77      0.79      0.78       585
       angry       0.84      0.81      0.82       585
       happy       0.86      0.88      0.87       593
    surprise       0.84      0.81      0.83       622
        fear       0.66      0.66      0.66       627
         sad       0.83      0.84      0.83       588

    accuracy                           0.80      3600
   macro avg       0.80      0.80      0.80      3600
weighted avg       0.80      0.80      0.80      3600



array([[461,  27,  18,  16,  45,  18],
       [ 34, 473,   9,  13,  38,  18],
       [ 12,   3, 520,   7,  35,  16],
       [ 18,   6,  13, 503,  60,  22],
       [ 63,  46,  35,  36, 416,  31],
       [ 13,  10,  13,  22,  34, 496]], dtype=int64)